In [1]:
import pandas as pd

df = pd.read_csv('merged_output.csv', dtype=str)

new_indicator = 'Agricultural water withdrawal as % of total renewable water resources'
years = [str(year) for year in range(1960, 2024)]

# Construcing new rows for each pair of country name and country code
countries = df[['Country Name', 'Country Code']].drop_duplicates()
new_rows = pd.DataFrame([
    {
        'Country Name': country,
        'Country Code': code,
        'Indicator Name': new_indicator,
        **{year: pd.NA for year in years}
    }
    for country, code in zip(countries['Country Name'], countries['Country Code'])
])

# Combining and sorting
df_combined = pd.concat([df, new_rows], ignore_index=True)
df_sorted = df_combined.sort_values(by=['Country Name', 'Country Code', 'Indicator Name'], kind='stable').reset_index(drop=True)

output_path = 'extended_output_sorted.csv'
df_sorted.to_csv(output_path, index=False)

df_sorted.groupby('Country Name').head(8).head(16)


,Country Name,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,AFG,Agricultural land (% of land area),NaN,57.878355794735,57.9550158686353,58.0316759425356,58.1160020238259,58.123668031216,58.1926620977263,...,58.123668031216,58.123668031216,58.123668031216,58.123668031216,58.2769881790166,58.2769881790166,58.7415482268525,58.7415482268525,58.7415482268525,NaN
1,Afghanistan,AFG,Agricultural water withdrawal as % of total re...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Afghanistan,AFG,"Agriculture, forestry, and fishing, value adde...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.1370413719574,20.6343227166798,25.7403140364582,26.4201990834448,22.0428967527575,25.7739707394105,29.9755825210461,33.5976188725451,33.7014323213923,34.5208917527827
3,Afghanistan,AFG,"Agriculture, forestry, and fishing, value adde...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4537457828.43798,3948217043.4989,4663262627.13517,4954700541.75017,3979453237.53176,4845363322.05338,5981905980.86851,4791018919.53499,4885778832.03434,5949003095.47994
4,Afghanistan,AFG,Fertilizer consumption (kilograms per hectare ...,NaN,0.143790849673203,0.142857142857143,0.141935483870968,0.141025641025641,0.141025641025641,0.1914486279515,...,10.1661124694377,13.3244919510625,13.6801462026135,20.4525288998571,17.3522286374134,10.9725,8.71449482692553,5.20257248690765,5.05506194916337,NaN
5,Afghanistan,AFG,Forest area (% of land area),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.85278199408184,1.85278199408184,1.85278199408184,1.85278199408184,1.85278199408184,1.85278199408184,1.85278199408184,1.85278199408184,1.85278199408184,NaN
6,Afghanistan,AFG,Permanent cropland (% of land area),NaN,0.153320147800622,0.153320147800622,0.153320147800622,0.160986155190654,0.168652162580685,0.183984177360747,...,0.213115005442865,0.222314214310903,0.277509467519127,0.323505511859313,0.331171519249345,0.340370728117382,0.340370728117382,0.340370728117382,0.340370728117382,NaN
7,Africa Eastern and Southern,AFE,Agricultural land (% of land area),NaN,40.4884524207293,40.4190804977299,40.4455406683479,40.4138108402359,40.3505782221657,40.3575953496702,...,45.9780444690021,46.0506179101141,46.1616694417186,46.2460594392602,46.3695581728235,46.4191789506879,46.4569755495826,46.5105662241913,46.541210005602,NaN
8,Africa Eastern and Southern,AFE,Agricultural water withdrawal as % of total re...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,Africa Eastern and Southern,AFE,"Agriculture, forestry, and fishing, value adde...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.9650929912911,12.8562394206678,12.9927693028971,12.446983160879,11.9139864893255,12.6275737797666,14.6442681024484,13.4044384367582,13.8407446717648,14.2318015672532


In [2]:
ext = pd.read_csv('extended_output_sorted.csv', dtype=str)

aqua = pd.read_csv('AQUASTAT Dissemination System.csv', dtype={'Area': str, 'Year': int, 'Value': float})

indicator = 'Agricultural water withdrawal as % of total renewable water resources'

# Filter for the indicator and average by (Area, Year)
aqua_filt = aqua[aqua['Variable'] == indicator]
aqua_agg = aqua_filt.groupby(['Area', 'Year'], as_index=False)['Value'].mean()

aqua_pivot = aqua_agg.pivot(index='Area', columns='Year', values='Value')   # Area as index, Year as columns
aqua_pivot.columns = aqua_pivot.columns.astype(str)      

years = [str(y) for y in range(1960, 2024)]

# Populate by matching Country Name and Year
mask = ext['Indicator Name'] == indicator
for yr in years:
    ext.loc[mask, yr] = ext.loc[mask, 'Country Name'].map(aqua_pivot.get(yr, pd.Series(dtype=float)))

output_path = 'extended_output_filled.csv'
ext.to_csv(output_path, index=False)

ext.loc[mask, ['Country Name', 'Country Code', 'Indicator Name'] + years].head(10)


,Country Name,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,Afghanistan,AFG,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.613807,30.613807,30.613807,30.613807,30.613807,30.613807,30.613807,30.613807,30.613807,NaN
8,Africa Eastern and Southern,AFE,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Africa Western and Central,AFW,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Albania,ALB,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.074172,2.119205,2.112583,2.31457,2.162252,2.291391,1.804636,1.86755,1.870861,NaN
29,Algeria,DZA,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.752864,53.244193,54.855576,57.178366,59.238307,61.298249,63.35819,61.129682,63.349619,NaN
36,American Samoa,ASM,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Andorra,AND,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Angola,AGO,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.098854,0.098854,0.098854,0.098854,0.098854,0.098854,0.098854,0.098854,0.098854,NaN
57,Antigua and Barbuda,ATG,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.461538,3.461538,3.461538,3.461538,3.461538,3.461538,3.461538,3.461538,3.461538,NaN
64,Arab World,ARB,Agricultural water withdrawal as % of total re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd
import pycountry
import pycountry_convert as pc

df = pd.read_csv('extended_output_filled.csv', dtype=str)

years = [str(y) for y in range(1960, 2024)]
df[years] = df[years].apply(pd.to_numeric, errors='coerce')

# ISO3-to-Continent mapping
def iso3_to_continent(iso3):
    try:
        iso2 = pycountry.countries.get(alpha_3=iso3).alpha_2
        code = pc.country_alpha2_to_continent_code(iso2)
        return {
            'AF': 'Africa',
            'AS': 'Asia',
            'EU': 'Europe',
            'NA': 'North America',
            'OC': 'Oceania',
            'SA': 'South America',
            'AN': 'Antarctica',
        }.get(code, 'Other')
    except:
        return 'Other'

df['Continent'] = df['Country Code'].apply(iso3_to_continent)

# Sum the monetary indicator by continent
sum_inds = ['Agriculture, forestry, and fishing, value added (current US$)']

sum_df = (df[df['Indicator Name'].isin(sum_inds)]
          .groupby(['Continent','Indicator Name'], as_index=False)[years]
          .sum(numeric_only=True))

# Average all other indicators by continent
mean_df = (df[~df['Indicator Name'].isin(sum_inds)]
           .groupby(['Continent','Indicator Name'], as_index=False)[years]
           .mean(numeric_only=True))

# Replace Country Name/Code with continent
agg = pd.concat([sum_df, mean_df], ignore_index=True)
agg['Country Name'] = agg['Continent']
agg['Country Code'] = ''

cols = ['Country Name','Country Code','Indicator Name'] + years
agg = agg[cols]
agg.to_csv('aggregated_by_continent.csv', index=False)

print("已生成：aggregated_by_continent.csv")
print(agg.head())


已生成：aggregated_by_continent.csv
    Country Name Country Code  \
0         Africa                
1           Asia                
2         Europe                
3  North America                
4        Oceania                

                                      Indicator Name          1960  \
0  Agriculture, forestry, and fishing, value adde...  5.039080e+09   
1  Agriculture, forestry, and fishing, value adde...  4.416795e+10   
2  Agriculture, forestry, and fishing, value adde...  6.095192e+09   
3  Agriculture, forestry, and fishing, value adde...  4.112402e+08   
4  Agriculture, forestry, and fishing, value adde...  0.000000e+00   

           1961          1962          1963          1964          1965  \
0  5.342556e+09  5.374349e+09  5.750809e+09  6.097495e+09  8.359606e+09   
1  4.914107e+10  5.003346e+10  5.653292e+10  6.326614e+10  6.807401e+10   
2  5.696093e+09  6.916855e+09  6.974673e+09  7.001966e+09  7.381315e+09   
3  2.524287e+08  2.610075e+08  4.762812e+08  4.8

In [6]:
import pandas as pd

agg = pd.read_csv('aggregated_by_continent.csv', dtype=str)

years = [str(y) for y in range(1960, 2024)]

agg[years] = agg[years].apply(pd.to_numeric, errors='coerce')

# Sort according to continents' names
agg_sorted = agg.sort_values(by='Country Name').reset_index(drop=True)

agg_sorted.to_csv('aggregated_by_continent_sorted.csv', index=False)

print(agg_sorted.head())


  Country Name Country Code  \
0       Africa          NaN   
1       Africa          NaN   
2       Africa          NaN   
3       Africa          NaN   
4       Africa          NaN   

                                      Indicator Name          1960  \
0  Agriculture, forestry, and fishing, value adde...  5.039080e+09   
1                Permanent cropland (% of land area)           NaN   
2                       Forest area (% of land area)           NaN   
3  Fertilizer consumption (kilograms per hectare ...           NaN   
4  Agricultural water withdrawal as % of total re...           NaN   

           1961          1962          1963          1964          1965  \
0  5.342556e+09  5.374349e+09  5.750809e+09  6.097495e+09  8.359606e+09   
1  2.268256e+00  2.290588e+00  2.307902e+00  2.332944e+00  2.375705e+00   
2           NaN           NaN           NaN           NaN           NaN   
3  1.331979e+01  1.427421e+01  1.576332e+01  1.654202e+01  1.629499e+01   
4           NaN  

In [7]:
import pandas as pd
import numpy as np
import pycountry
import pycountry_convert as pc

df = pd.read_csv('extended_output_filled.csv', dtype=str)

years = [str(y) for y in range(1960, 2024)]
df[years] = df[years].apply(pd.to_numeric, errors='coerce')

def iso3_to_continent(iso3):
    try:
        iso2 = pycountry.countries.get(alpha_3=iso3).alpha_2
        code = pc.country_alpha2_to_continent_code(iso2)
        return {
            'AF': 'Africa','AS': 'Asia','EU': 'Europe',
            'NA': 'North America','OC': 'Oceania',
            'SA': 'South America','AN': 'Antarctica',
        }[code]
    except:
        return 'Other'

df['Continent'] = df['Country Code'].apply(iso3_to_continent)

sum_inds = ['Agriculture, forestry, and fishing, value added (current US$)']

# Strict - compute only if all values are present, otherwise return NaN
def sum_if_complete(s): return s.sum() if s.notna().all() else np.nan
def mean_if_complete(s): return s.mean() if s.notna().all() else np.nan

# Sum and average by continent
sum_df = df[df['Indicator Name'].isin(sum_inds)]
sum_df = sum_df.groupby(['Continent','Indicator Name'], as_index=False)[years].agg(sum_if_complete)

mean_df = df[~df['Indicator Name'].isin(sum_inds)]
mean_df = mean_df.groupby(['Continent','Indicator Name'], as_index=False)[years].agg(mean_if_complete)

agg = pd.concat([sum_df, mean_df], ignore_index=True)
agg['Country Name'] = agg['Continent']
agg['Country Code'] = ''

cols = ['Country Name','Country Code','Indicator Name'] + years
agg = agg[cols]
agg.to_csv('aggregated_by_continent_strict.csv', index=False)

print("已生成：aggregated_by_continent_strict.csv")
print(agg.head())


已生成：aggregated_by_continent_strict.csv
    Country Name Country Code  \
0         Africa                
1           Asia                
2         Europe                
3  North America                
4        Oceania                

                                      Indicator Name  1960  1961  1962  1963  \
0  Agriculture, forestry, and fishing, value adde...   NaN   NaN   NaN   NaN   
1  Agriculture, forestry, and fishing, value adde...   NaN   NaN   NaN   NaN   
2  Agriculture, forestry, and fishing, value adde...   NaN   NaN   NaN   NaN   
3  Agriculture, forestry, and fishing, value adde...   NaN   NaN   NaN   NaN   
4  Agriculture, forestry, and fishing, value adde...   NaN   NaN   NaN   NaN   

   1964  1965  1966  ...  2014  2015  2016  2017  2018  2019  2020  2021  \
0   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   Na